In [2]:
from engine.grammar import Parser
from razdel import tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

gramm = """
<CONJ> ::= CONJ | COMMA <CONJ>
<S> ::= [nomn:NOUN,NPRO]+ | ADJF <S> 
<ASPECT> ::= [PREP,ADVB,ADJF]+ [NOUN,<S>] | <ASPECT> <CONJ> <ASPECT>
<PARTICIPLE> ::= COMMA PRTF [ADVB,NOUN,<ASPECT>]+ COMMA? | PRTF 
<TRANSGRESSIVE> ::= COMMA GRND [ADVB,NOUN,<ASPECT>]+ COMMA? | GRND
<S> ::= <S> [<PARTICIPLE>,<TRANSGRESSIVE>] | [<PARTICIPLE>,<TRANSGRESSIVE>] <S> | <S> <CONJ> <S>
<VP> ::= [VERB,ADJS,INFN,PRTS,COMP]+ | [ADVB,NOUN,<ASPECT>,<PARTICIPLE>,<TRANSGRESSIVE>]+ <VP> 
<VP> ::= <VP> [ADVB,NOUN,<ASPECT>,<PARTICIPLE>,<TRANSGRESSIVE>]+ | <VP> <CONJ> <VP>

<SEN> ::= [<VP>,<S>]+

"""

def precr(token):
    if token == ',':
        return 'COMMA'
    t = morph.parse(token)[0].tag
    if (t.POS == 'NOUN') and (t.case == 'nomn'):
        return 'nomn:NOUN'
    else:
        return str(t.POS)


grammatics = Parser(rules=gramm, 
                    tokenizer=lambda text: [token.text for token in tokenize(text)],
                    preprocessor=precr)

In [3]:
%%time

tree = grammatics.parse('Я вышел холодной осенью на улицу')
tree

Wall time: 13.6 ms


[Я вышел холодной осенью на улицу]

In [4]:
tree[0].print_tree()

          <SEN>                  
            |                     
                                 
  __________|______               
 |                <VP>           
 |                 |              
 |                               
 |     ____________|________      
<S>  <VP>       <ASPECT> <ASPECT>
 |    |            |        |     
 Я  вышел       холодной    на   
 |    |            |        |     
...  ...         осенью   улицу  



In [91]:
%%time

tree = grammatics.parse('Мальчик купил пельмени и пошёл домой, а девочка посмотрела в телефон и заплакала')
tree

Wall time: 82.9 ms


[Мальчик купил пельмени и пошёл домой,
 , а,
 девочка посмотрела в телефон и заплакала]

In [60]:
tree[0].print_tree()

                      <SEN>                        
                        |                           
                                                   
    ____________________|_____                      
   |                         <VP>                  
   |                          |                     
   |                                               
   |               ___________|___________          
   |             <VP>         |          <VP>      
   |              |           |           |         
   |                          |                    
   |        ______|_____      |       ____|_____    
  <S>      |           <VP> <CONJ>   |         <VP>
   |       |            |     |      |          |   
Мальчик    |          купил   и      |        пошёл
   |       |            |     |      |          |   
  ...   пельмени       ...   ...   домой       ... 



In [92]:
%%time

tree = grammatics.parse('Я и Петя, и Ваня вышли во двор, а Коля остался дома')
tree

Wall time: 50.1 ms


[Я и Петя , и Ваня вышли во двор, , а, Коля остался дома]

In [62]:
tree[0].print_tree()

                <SEN>                          
                  |                             
                                               
             _____|________________             
           <S>                     |           
            |                      |            
                                   |           
       _____|___________           |            
     <S>        <CONJ>  |         <VP>         
      |           |     |          |            
                  ,     |                      
  ____|_____      |     |      ____|______      
<S> <CONJ> <S>  <CONJ> <S>   <VP>      <ASPECT>
 |    |     |     |     |     |           |     
 Я    и    Петя   и    Ваня вышли         во   
 |    |     |     |     |     |           |     
...  ...   ...   ...   ...   ...         двор  



In [63]:
tree[-1].print_tree()

     <SEN>             
       |                
                       
  _____|____            
 |         <VP>        
 |          |           
 |                     
 |      ____|______     
<S>    |          <VP> 
 |     |           |    
Коля   |        остался
 |     |           |    
...   дома        ...  



In [93]:
%%time

tree = grammatics.parse('Вскоре после восхода набежала туча и брызнул дождь')
tree

Wall time: 20.4 ms


[Вскоре после восхода набежала туча, и, брызнул дождь]

In [68]:
tree[0].print_tree()

                       <SEN>              
                         |                 
                                          
                   ______|_____________    
                 <VP>                  |  
                  |                    |   
                                       |  
          ________|_____________       |   
      <ASPECT>                 <VP>   <S> 
         |                      |      |   
       Вскоре                набежала туча
   ______|________              |      |   
после          восхода         ...    ... 



In [94]:
%%time

tree = grammatics.parse('Александр стоял у дома, куря сигарету')
tree

Wall time: 3.16 ms


[Александр стоял у дома , куря сигарету]

In [95]:
tree[0].print_tree()

          <SEN>                                       
            |                                          
                                                      
     _______|______                                    
    |             <VP>                                
    |              |                                   
    |                                                 
    |        ______|_________________                  
   <S>     <VP> <ASPECT>      <TRANSGRESSIVE>         
    |       |      |                 |                 
Александр стоял    у                 ,                
    |       |      |       __________|___________      
   ...     ...    дома   куря                 сигарету



In [96]:
%%time

tree = grammatics.parse('Было холодно, падал снег')
tree

Wall time: 19.2 ms


[Было холодно, ',', падал снег]

In [5]:
simplify = lambda text: [unit for unit in grammatics.parse(text) \
                         if hasattr(unit, 'label') and (unit.label != '<CONJ>')]

In [6]:
%%time

simplify('Было холодно, падал снег')

Wall time: 41.6 ms


[Было холодно, падал снег]

In [8]:
treebank = simplify('Елка, стоявшая у стены, внезапно обвалилась, и мы пошли гулять')
treebank

[Елка , стоявшая у стены , внезапно обвалилась, мы пошли гулять]

In [12]:
treebank[0].print_tree()

                     <SEN>                        
                       |                           
                                                  
                _______|____________________       
              <S>                           |     
               |                            |      
                                            |     
  _____________|_______                     |      
 |                <PARTICIPLE>             <VP>   
 |                     |                    |      
 |                     ,                 внезапно 
 |       ______________|__________          |      
<S>     |              |       <ASPECT>    <VP>   
 |      |              |          |         |      
Елка    |              |          у     обвалилась
 |      |              |          |         |      
...  стоявшая          ,        стены      ...    



In [107]:
%%time

simplify('Курящий дворник посмотрел на меня сердито, а я испугался')

Wall time: 30.6 ms


['Курящий дворник посмотрел на меня сердито', 'я испугался']